# Create LDA for Corpus<a id='top'></a>

1. Create a JSON file for each blog or social media profile of your corpus; each entry or post is a line in a JSON file. One way to do this is to crawl websites using [scrapy](https://scrapy.org) with these flags: "-o result.json -t json" (see [sample crawlers](./scripts/scraper/spiders) and [example item file](./scripts/scraper/items.py)). An example JSON file is [here](./scripts/example.json).
2. Prepare corpus according to your needs, e.g. filter stopwords. My [example](./scripts/text.py) is tailored to Russian texts. It removes all non-cyrillic characters, removes all words which are not nouns and sets all nouns into first person singular using POS tagging.
3. [Create LDA model for the corpus](#create)
4. [Compute topic distribution for corpus](#compute)
5. [Explore corpus](corpus.ipynb) (different notebook)

Due to copyright reasons I cannot publish the scraped raw data. The results of the smoothing process in step 2 are [here](./corpus/); they are used in the examples below.

In [1]:
import os
import sys
from gensim import corpora, models
import logging
import errno
import pandas as pd
from dateutil import parser
import pytz
import numpy as np
import json

## Create LDA model for corpus<a id='create'></a>

<b>NB: Creating the model might take up to 3 hours</b>

[Back to top](#top)

In [ ]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

corpus = []

model_name = "model"
result_path = "results"
corpus_path = "corpus"

try:
    os.makedirs(result_path)
except OSError as exception:
    if exception.errno != errno.EEXIST:
        raise
        
for json_file in os.listdir(corpus_path):
    print("File: ", json_file)

    if json_file.endswith(".json"):
        # get data
        json_data = open(os.path.join(corpus_path, json_file))
        data = json.load(json_data)
        json_data.close()

        for entry in data:
            corpus.append(entry["text"].split(" "))

print("File extraction complete.")

try:
    dictionary = corpora.Dictionary.load(os.path.join(result_path, model_name + ".dict"))
except IOError:
    dictionary = corpora.Dictionary(corpus)
    dictionary.save(os.path.join(result_path, model_name + ".dict"))

corpus = [dictionary.doc2bow(text) for text in corpus]

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=50, alpha='auto', eval_every=5, passes=20)

start = 1
while os.path.isfile(os.path.join(result_path, model_name + "-" +str(start)+ ".lda")):
    start += 1

lda.save(os.path.join(result_path, model_name + "-" +str(start)+ ".lda"))

print("LDA saved as", os.path.join(result_path, model_name + "-" +str(start)+ ".lda"))

## Compute topic distribution for corpus<a id='compute'></a>

[Back to top](#top)

In [ ]:
# entries published after max_date are ignored
utc = pytz.UTC
max_date = parser.parse("2014-12-31 23:59:59")
max_date_utc = utc.localize(parser.parse("2014-12-31 23:59:59"))

# change lda_name to "pos_tags" for thesis corpus
corpus_path = "corpus"
result_path = "results"
model_name = "model"
topics_name = "topics"

number = 0
for f in os.listdir(result_path):
    try:
        number = max(number, int(f.split(model_name+"-")[1].split(".lda")[0]))
    except IndexError:
        continue
if number > 0:
    file_name = model_name + "-" + str(number) + ".lda"
else: 
    file_name = model_name + ".lda"

# load LDA model and dictionary
dictionary = corpora.Dictionary.load(os.path.join(result_path, model_name + ".dict"))
model = models.LdaModel.load(os.path.join(result_path, file_name))

# new fields for compatibility, default values from
# https://radimrehurek.com/gensim/models/ldamodel.html
try:
    x = model.minimum_probability
except AttributeError:
    model.minimum_probability = 0.01
    model.minimum_phi_value = 0.01
    model.per_word_topics = False
    model.random_state = np.random.RandomState()

columns = ['group', 'url', 'date', 'comment_count', 'words']
columns.extend([str(topic) for topic in range(model.num_topics)])

result = []

# sort files
for json_file in sorted(os.listdir(corpus_path)):

    print("File: ", json_file)

    if json_file.endswith(".json"):
        # get data
        with open(os.path.join(corpus_path, json_file)) as json_data:
            data = json.load(json_data)

        removed = 0
        too_short = 0

        for entry in data:
            # check if entry is within data range
            try:
                date = parser.parse(entry["date"])
                try:
                    if date > max_date:
                        removed += 1
                        continue
                except TypeError:
                    if date > max_date_utc:
                        removed += 1
                        continue
            except ValueError:
                print("Wrong format", entry["date"])

            # get topic distribution for entry
            line = {}
            text = entry["text"].split(" ")
                
            # filter too short entries
            if len(text) < 5:
                too_short += 1
                continue

            topics = [0] * model.num_topics
            for (topic, prop) in model[dictionary.doc2bow(text)]:
                topics[topic] = prop
            line["group"] = json_file.split(".json")[0]
            line["url"] = entry["url"]
            line["date"] = entry['date']
            line["words"] = len(text)
            line["comment_count"] = entry["comment_count"]
            for counter in range(len(topics)):
                line[str(counter)] = topics[counter]
            result.append(line)

        print("Total number of entries:", len(data))
        print("Removed because of date: ", removed)
        print("Removed because too short: ", too_short)
        print("Remaining:", (len(data) - removed - too_short))
            
frame = pd.DataFrame(result)
frame = frame[columns]
start = 1
while os.path.isfile(os.path.join(result_path, topics_name + "-" +str(start)+ ".json")):
    start += 1

frame.to_json(os.path.join(result_path, topics_name + "-" + str(start) + ".json"), orient='split')
print ("Created", os.path.join(result_path, topics_name + "-" + str(start) + ".json"))